# Twitter Named Entity Recognition Case Study

### About
Twitter is a microblogging and social networking service on which users post and interact with messages known as "tweets". Every second, on average, around 6,000 tweets are tweeted on Twitter, corresponding to over 350,000 tweets sent per minute, 500 million tweets per day.

### Problem statement 
Twitter wants to automatically tag and analyze tweets for better understanding of the trends and topics without being dependent on the hashtags that the users use. Many users do not use hashtags or sometimes use wrong or mis-spelled tags, so they want to completely remove this problem and create a system of recognizing important content of the tweets.

### Objective
Named Entity Recognition (NER) is an important subtask of information extraction that seeks to locate and recognise named entities.
We need to train models that will be able to identify the various named entities.

### Data
Dataset is annotated with 10 fine-grained NER categories: person, geo-location, company, facility, product,music artist, movie, sports team, tv show and other. Dataset was extracted from tweets and is structured in CoNLL format., in English language. Containing in Text file format.
The CoNLL format is a text file with one word per line with sentences separated by an empty line. The first word in a line should be the word and the last word should be the label.

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
root_path = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
data_path = os.path.join(root_path,'data')
train_data_path = os.path.join(data_path,'wnut 16.txt.conll')
test_data_path = os.path.join(data_path,'wnut 16test.txt.conll')

## Getting the data

In [3]:
# reading the training file
with open(train_data_path,'r') as f:
    train_raw = f.read()
with open(test_data_path,'r') as f:
    test_raw = f.read()

In [4]:
# creating a function to format the data
def extract_ner_from_conll(conll_data):
    # Split the data into sentences based on empty lines
    sentences = [sentence.strip() for sentence in conll_data.strip().split('\n\n')]
    ner_data = []

    for sentence in sentences:
        tokenised_sentence = []
        for token_entity in sentence.split('\n'):
            token, entity = token_entity.split('\t')
            tokenised_sentence.append((token,entity))
        ner_data.append(tokenised_sentence)

    return ner_data

In [5]:
# preprocessing the raw files
train_data = extract_ner_from_conll(train_raw)
test_data = extract_ner_from_conll(test_raw)

In [6]:
# checking sentences after preprocessing
print(train_data[0])

[('@SammieLynnsMom', 'O'), ('@tg10781', 'O'), ('they', 'O'), ('will', 'O'), ('be', 'O'), ('all', 'O'), ('done', 'O'), ('by', 'O'), ('Sunday', 'O'), ('trust', 'O'), ('me', 'O'), ('*wink*', 'O')]


## EDA

In [7]:
# number of words in the vocabulary and lenght of sentences in the training data

sentence_lenghts = list()
word_set = set()
for sentence in train_data:
    sentence_lenghts.append(len(sentence))
    for word in sentence:
        word_set.add(word[0])
        
NUM_WORDS = len(word_set)+2 # +2 to include padding and out of vocabulary
print(f"Number of unique words in training data (including padding and OOV token) = {NUM_WORDS}")
print(f"Maximum sentence length = {max(sentence_lenghts)}")
print(f"Minimum sentence length = {min(sentence_lenghts)}")

Number of unique words in training data (including padding and OOV token) = 10588
Maximum sentence length = 39
Minimum sentence length = 1


In [8]:
# since the max sentence length if 39, we will take a length of 45 in our model to incorporate for edge cases in inference
SENTENCE_LENGTH = 45
# we will keep the embedding dimensions to be 50 since the number of datapoints is small
EMBEDDING_DIMS = 50

In [9]:
# number of entities
entity_set = set()
for sentence in train_data:
    for word in sentence:
        entity_set.add(word[1])
        
NUM_ENTITIES = len(entity_set)
print(f"Number of unique entities in training data = {NUM_ENTITIES}")

Number of unique entities in training data = 21


## Data preparation

In [10]:
# create a function to prepare the data to be fed into the model
# --> punctuation should be handled along with their entities
def prepare_data(text_data):
    
    # initialize empty lists for sentences and entities
    sentences = []
    entities = []
    
    for sentence in text_data:
        
        # initialize empty lists for sentence text and corresponding entities
        word_list = []
        entity_list = []
        
        for token in sentence:
            word_list.append(token[0])
            entity_list.append(token[1])
        
        sentences.append(word_list)
        entities.append(entity_list)
    
    # create a single string for each sentence and entity by joining elements with whitespace
    sentences = np.array([' '.join(sentence) for sentence in sentences])
    entities = np.array([' '.join(entity) for entity in entities])
    
    return (sentences,entities)

In [11]:
# call the prepare_data function on the train and test data
xtrain, ytrain = prepare_data(train_data)
xtest, ytest = prepare_data(test_data)

In [12]:
# checking sentences are conversion
print(xtrain[0])
print(ytrain[0])

@SammieLynnsMom @tg10781 they will be all done by Sunday trust me *wink*
O O O O O O O O O O O O


## Bidirectional LSTM + CRF

In [13]:
from tensorflow.keras.layers import Input, TextVectorization, Embedding, Bidirectional, LSTM, TimeDistributed, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow_addons.layers import CRF
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

In [14]:
# adapt the vectorizer layer before modeling
sentence_tokenizer = TextVectorization(max_tokens=NUM_WORDS, output_sequence_length=SENTENCE_LENGTH, standardize='lower', name='sentence_tokenizer')
sentence_tokenizer.adapt(xtrain)

In [15]:
# define a function to build and compile model
def build_lstm_model(name='bi-lstm+crf'):
    
    # input layer for getting sentences
    sentence_input = Input(shape=(1,), dtype=tf.string, name='sentence_input')
    
    # tokenizing the sentence
    x = sentence_tokenizer(sentence_input)
    
    # creating embeddings for each token in sentence
    embeddings = Embedding(
        input_dim = NUM_WORDS,
        output_dim = EMBEDDING_DIMS,
        mask_zero = True,
        name = 'word_embedding'
    )(x)
    
    # stacking two bidirectional LSTMs
    output_sequence = Bidirectional(LSTM(40, return_sequences=True), name='lstm_1')(embeddings)
    output_sequence = Bidirectional(LSTM(40, return_sequences=True), name='lstm_2')(output_sequence)
    
    # passing the sequence through dense layer to compress the information
    dense_sequence = TimeDistributed(Dense(25, activation='relu'), name='dense')(output_sequence)
    
    # passing the dense sequences through crf layer
    predicted_sequence, potentials, sequence_length, crf_kernel = CRF(NUM_ENTITIES, name='crf')(dense_sequence)
    
    # define the train model
    training_model = Model(sentence_input, potentials)
    
    # compile the model
    training_model.compile(
        loss=SigmoidFocalCrossEntropy(),
        optimizer='adam'
    )
    
    # create an inference model
    inference_model = Model(sentence_input, predicted_sequence)
    
    return training_model, inference_model

In [16]:
# creating the traing and inferencing model
lstm_training_model, lstm_inference_model = build_lstm_model()

In [17]:
lstm_training_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sentence_input (InputLayer  [(None, 1)]               0         
 )                                                               
                                                                 
 sentence_tokenizer (TextVe  (None, 45)                0         
 ctorization)                                                    
                                                                 
 word_embedding (Embedding)  (None, 45, 50)            529400    
                                                                 
 lstm_1 (Bidirectional)      (None, 45, 80)            29120     
                                                                 
 lstm_2 (Bidirectional)      (None, 45, 80)            38720     
                                                                 
 dense (TimeDistributed)     (None, 45, 25)            2025  

In [172]:
# prepare the entity data

# tokenizing the entities
entity_tokenizer = TextVectorization(max_tokens=NUM_ENTITIES, output_sequence_length=SENTENCE_LENGTH, standardize='lower', name='entity_tokenizer')
entity_tokenizer.adapt(ytrain)
ytrain_tokenized = entity_tokenizer(ytrain)
ytest_tokenized = entity_tokenizer(ytest)

# one hot encoding the entitiy tokens
entity_ohe = Lambda(lambda x: tf.one_hot(x, NUM_ENTITIES))
ytrain_tokenized = entity_ohe(ytrain_tokenized)
ytest_tokenized = entity_ohe(ytest_tokenized)

In [150]:
history = lstm_training_model.fit(xtrain, ytrain_tokenized, epochs=20, batch_size=16)

Epoch 1/20
150/150 [==============================] - 5s 35ms/step - loss: 0.1776
Epoch 2/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0876
Epoch 3/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0466
Epoch 4/20
150/150 [==============================] - 5s 33ms/step - loss: 0.0281
Epoch 5/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0191
Epoch 6/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0137
Epoch 7/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0110
Epoch 8/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0087
Epoch 9/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0072
Epoch 10/20
150/150 [==============================] - 5s 33ms/step - loss: 0.0062
Epoch 11/20
150/150 [==============================] - 5s 34ms/step - loss: 0.0052
Epoch 12/20
150/150 [==============================] - 5s 33ms/step - loss: 0.0043
Epoch 13/20
1

In [174]:
lstm_training_model.evaluate(xtest,ytest_tokenized)

121/121 [==============================] - 1s 12ms/step - loss: 0.5763


0.5762874484062195

In [20]:
# implement the validation data somehow

### Inferencing the Bi-LSTM+CRF model

In [24]:
print(xtest[0])
print(ytest[0])
print(len(xtest[0]))
print(len(ytest[0]))

New Orleans Mother 's Day Parade shooting . One of the people hurt was a 10-year-old girl . WHAT THE HELL IS WRONG WITH PEOPLE ?
B-other I-other I-other I-other I-other I-other O O O O O O O O O O O O O O O O O O O O
128
87


In [45]:
ytest[[2]][0].split()

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-movie', 'O', 'O']

In [151]:
# function to infer prediction from a text in xtest randomly
def infer_lstm():
    idx = np.random.choice(range(len(xtest)),1,replace=False)
    text = xtest[idx]
    pred_labels = lstm_inference_model.predict(text)
    act_labels = ytest[idx]
    text_len = len(text[0].split())
    
    pred_labels = np.asarray(entity_tokenizer.get_vocabulary())[list(pred_labels[0])]
    result = pd.DataFrame(
        {
            'text':text[0].split(),
            'actual_labels':act_labels[0].split(),
            'predicted_labels':pred_labels[:text_len]
        }
    )
    display(result)

In [166]:
infer_lstm()

1/1 [==============================] - 0s 25ms/step


,text,actual_labels,predicted_labels
0,Gunman,O,o
1,identified,O,o
2,in,O,o
3,shooting,O,o
4,deaths,O,o
5,of,O,o
6,4,O,o
7,Marines,O,o
8,at,O,o
9,Tennessee,B-geo-loc,b-other
